<a href="https://colab.research.google.com/github/3zero69/-/blob/main/1114_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%B6%84%EB%A5%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-11-14 10:08:26--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-11-14 10:08:26--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 74.125.20.113, 74.125.20.139, 74.125.20.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.20.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5kgi3m0h94bgc60f3d4rqf14kklh4efu/1668420450000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download&uuid=217a33a0-f7c7-45f1-b079-8301f5691d94 [following]
--2022-11-14 10:08:26--  https://doc-10-10-docs.google

1. 필요한 라이브러리를 가져옵니다.

In [2]:
import pandas as pd
import lightgbm as lgbm

2. 데이터를 불러옵니다.

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
smpl_sub = pd.read_csv('data/sample_submission.csv')

In [4]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [5]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


3. 변수를 변환합니다. 

In [33]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.inverse_transform(train['type'])
test['type'] = enc.inverse_transform(test['type'])

In [34]:
train

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1,1
5493,6,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0,1
5494,7,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3,1
5495,5,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5,1


In [6]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.
train['type'] = train['type'].map({'white':0, 'red':1}).astype(int)
test['type'] = test['type'].map({'white':0, 'red':1}).astype(int)

In [7]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0


In [8]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0


In [9]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.

train_x = train.drop(columns = ['index'])
train_x = train.drop(columns = ['quality'])

train_y = train['quality']

test_x = train.drop(columns = ['index'])

In [10]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(5497, 13)
(5497,)
(5497, 13)


In [11]:
train_x.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0
1,1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1
2,2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0
3,3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0
4,4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0


In [12]:
train_y.head()

0    5
1    5
2    5
3    6
4    6
Name: quality, dtype: int64

In [13]:
test_x.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0


4. LightGBM 모델을 생성 및 훈련합니다.

In [14]:
from lightgbm import LGBMClassifier

lgbm_wrapper=LGBMClassifier()
lgbm_wrapper.fit(train_x,train_y)

LGBMClassifier()

5. 생성한 모델로 예측 데이터를 생성합니다.

In [15]:
y_pred = lgbm_wrapper.predict(test_x)

In [16]:
y_pred.shape

(5497,)

6. 제출파일을 생성합니다.

In [17]:
smpl_sub.head

<bound method NDFrame.head of      index  quality
0        0        0
1        1        0
2        2        0
3        3        0
4        4        0
..     ...      ...
995    995        0
996    996        0
997    997        0
998    998        0
999    999        0

[1000 rows x 2 columns]>

In [30]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
y_pred = smpl_sub['quality'] 

In [19]:
smpl_sub.head

<bound method NDFrame.head of      index  quality
0        0        0
1        1        0
2        2        0
3        3        0
4        4        0
..     ...      ...
995    995        0
996    996        0
997    997        0
998    998        0
999    999        0

[1000 rows x 2 columns]>

In [35]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [20]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [21]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [ ]:
# 원핫 인코딩 (pd.get_dummies())

train_x = pd.get_dummies(train)
test_x = pd.get_dummies(test)

train_x.head()

In [25]:
# 모델 정의
model = XGBClassifier()

In [26]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수

X = train_x.drop('quality',axis= 1)
y = train_x['quality']

# fit 메소드를 이용해 모델 학습
model.fit(X,y)

XGBClassifier(objective='multi:softprob')

In [27]:
# predict 메소드와 test_one 데이터를 이용해 품질 예측
pred = model.predict(test_x)

In [28]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred

submission.head()

,index,quality
0,0,6
1,1,5
2,2,5
3,3,6
4,4,6


In [29]:
# 정답파일 내보내기
submission.to_csv('xgb_pred.csv',index = False)